In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from os import listdir
from os.path import isfile, join

onlyfiles = [f for f in listdir("./data") if isfile(join("./data", f))]

So pick 456

In [ ]:
indmax = int(np.radians(90 - 5) / np.pi * 1000)
indmax

In [ ]:
indmin = int(np.radians(90 - 45) / np.pi * 1000)
indmin

In [ ]:
periods = []
FFs = []
S11s = []
wav_0 = []
wav_1 = []
Prs = []
Pnorms = []
farfield_angles = []
Pnorms = []
Prs = []

for file in onlyfiles:
    dbfile = open("./data/" + file, "rb")
    db = pickle.load(dbfile)

    farfield_angles_loc = db["farfield_angles"]

    # Far field global
    farfield_powers = db["farfield_power"]

    Ex = farfield_powers[:, 0]
    Ey = farfield_powers[:, 1]
    Ez = farfield_powers[:, 2]
    Hx = farfield_powers[:, 3]
    Hy = farfield_powers[:, 4]
    Hz = farfield_powers[:, 5]
    Ex = np.conj(Ex)
    Ey = np.conj(Ey)
    Ez = np.conj(Ez)
    Px = np.real(np.multiply(Ey, Hz) - np.multiply(Ez, Hy))
    Py = np.real(np.multiply(Ez, Hx) - np.multiply(Ex, Hz))
    Pz = np.real(np.multiply(Ex, Hy) - np.multiply(Ey, Hx))
    Pr = np.sqrt(np.square(Px) + np.square(Py))
    Pnorm = Pr / np.max(Pr)

    for ind in range(indmin, indmax):
        # Angle
        farfield_angles.append(90 - np.degrees(farfield_angles_loc[ind]))

        # Parameters
        params = db["params"]
        periods.append(params["a"])
        FFs.append(params["FF"])

        # Waveguide port
        res_waveguide = db["res_waveguide"]
        wav_0.append(res_waveguide.alpha[0, 0, 0])
        wav_1.append(res_waveguide.alpha[0, 0, 1])
        S11s.append(
            np.abs(res_waveguide.alpha[0, 0, 1]) ** 2
            / np.abs(res_waveguide.alpha[0, 0, 0]) ** 2
        )

        Prs.append(Pr[int(ind)])
        Pnorms.append(Pnorm[int(ind)])

    dbfile.close()

In [ ]:
import pandas as pd

dict_arr = {
    "period": periods,
    "FF": FFs,
    "angle": farfield_angles,
    "S11": S11s,
    "wav_0": wav_0,
    "wav_1": wav_1,
    "Prs": Prs,
    "Pnorms": Pnorms,
}
df = pd.DataFrame(dict_arr)
df

In [ ]:
def neff(FF):
    return 1.93 * FF + (1 - FF)


def angle(a, neff):
    return np.degrees(neff - np.arcsin(0.635 / a))

In [ ]:
np.max(np.abs(df["Prs"].to_numpy()))

In [ ]:
df.loc[df["Prs"] == np.max(df["Prs"].to_numpy())]

In [ ]:
import os
import re

for filename in os.listdir("./data/"):
    if re.match("\*a_0.96_FF_0.85\*", filename):
        file = filename

dbfile = open("./data/" + file, "rb")
db = pickle.load(dbfile)
dbfile.close()

farfield_power = db["farfield_power"]
farfield_angles = db["farfield_angles"]

Ex = farfield_power[:, 0]
Ey = farfield_power[:, 1]
Ez = farfield_power[:, 2]
Hx = farfield_power[:, 3]
Hy = farfield_power[:, 4]
Hz = farfield_power[:, 5]
Ex = np.conj(Ex)
Ey = np.conj(Ey)
Ez = np.conj(Ez)
Px = np.real(np.multiply(Ey, Hz) - np.multiply(Ez, Hy))
Py = np.real(np.multiply(Ez, Hx) - np.multiply(Ex, Hz))
Pz = np.real(np.multiply(Ex, Hy) - np.multiply(Ey, Hx))
Pr = np.sqrt(np.square(Px) + np.square(Py))
Pnorm = Pr / np.max(Pr)

fig, ax = plt.subplots(subplot_kw={"projection": "polar"})
ax.plot(farfield_angles, Pnorm)
ax.set_rmax(1)
ax.set_rticks([0.25, 0.5, 0.75, 1])  # Less radial ticks
ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
ax.grid(True)

# ax.set_title("A line plot on a polar axis", va='bottom')
plt.show()

In [ ]:
import meep as mp
import math

resolution = 32  # pixels/unit length (1 um)

hSiN = 0.44
hSiO2 = 3.2
hSi = 1
hair = 4

dgrat = 0.96 * 0.85
dgap = 0.96 * (1 - 0.85)
a = dgrat + dgap
N = 20
N = N + 1
dtaper = 12

dbuffer = 0.5
dpml = 1

# Fiber parameters, from SMF-633-4/125-1-L or PMF-633-4/125-0.25-L
fiber_core = 4
fiber_clad = 120
fiber_angle = 8
fiber_angle = np.radians(fiber_angle)
fiber_xposition = 0
fiber_air_gap = 1
hfiber = 3
haircore = 2
hfiber_geom = 100  # Some large number to make fiber extend into PML
# Index from 0.10 @ 633 nm numerical aperture
# cladding 635 nm real index, From NA, n=1.4535642400664652
nClad = 1.4535642400664652
Clad = mp.Medium(index=nClad)
# Pure fused silica core core 635 nm real index (will be SiO2 below)

# MEEP's computational cell is always centered at (0,0), but code has beginning of grating at (0,0)
sxy = 2 * dpml + dtaper + a * N + 2 * dbuffer  # sx here
sz = 2 * dbuffer + hSiO2 + hSiN + hair + hSi + 2 * dpml  # sy here
# comp_origin_x = dpml + dbuffer + dtaper
comp_origin_x = 0
# meep_origin_x = sxy/2
# x_offset = meep_origin_x - comp_origin_x
x_offset = 0
# comp_origin_y = dpml + hSi + hSiO2 + hSiN/2
comp_origin_y = 0
# meep_origin_y = sz/2
# y_offset = meep_origin_y - comp_origin_y
y_offset = 0

# x_offset_vector = mp.Vector3(x_offset,0)
# offset_vector = mp.Vector3(x_offset, y_offset)
offset_vector = mp.Vector3(0, 0, 0)

# Si3N4 635 nm real index
nSiN = 2.0102
SiN = mp.Medium(index=nSiN)
# SiO2 635 nm real index
nSiO2 = 1.4569
SiO2 = mp.Medium(index=nSiO2)
# Si substrate 635 nm complex index, following https://meep.readthedocs.io/en/latest/Materials/#conductivity-and-complex
# eps = 15.044 + i*0.14910
Si = mp.Medium(epsilon=15.044, D_conductivity=2 * math.pi * 0.635 * 0.14910 / 15.044)

# We will do x-z plane simulation
cell_size = mp.Vector3(sxy, sz)

geometry = []

# Fiber (defined first to be overridden)

# Core
# fiber_offset = mp.Vector3(fiber_xposition + extrax, hSiN/2 + hair + haircore + extray) - offset_vector
geometry.append(
    mp.Block(
        material=Clad,
        center=mp.Vector3(x=fiber_xposition) - offset_vector,
        size=mp.Vector3(fiber_clad, hfiber_geom),
        e1=mp.Vector3(x=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
        e2=mp.Vector3(y=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
    )
)
geometry.append(
    mp.Block(
        material=SiO2,
        center=mp.Vector3(x=fiber_xposition) - offset_vector,
        size=mp.Vector3(fiber_core, hfiber_geom),
        e1=mp.Vector3(x=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
        e2=mp.Vector3(y=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
    )
)

# air
geometry.append(
    mp.Block(
        material=mp.air,
        center=mp.Vector3(0, haircore / 2) - offset_vector,
        size=mp.Vector3(mp.inf, haircore),
    )
)

# waveguide
geometry.append(
    mp.Block(
        material=SiN,
        center=mp.Vector3(0, 0) - offset_vector,
        size=mp.Vector3(mp.inf, hSiN),
    )
)

# grating etch
for n in range(0, N):
    geometry.append(
        mp.Block(
            material=mp.air,
            center=mp.Vector3(n * a + dgap / 2, 0) - offset_vector,
            size=mp.Vector3(dgap, hSiN),
        )
    )

geometry.append(
    mp.Block(
        material=mp.air,
        center=mp.Vector3(sxy - comp_origin_x - 0.5 * (dpml + dbuffer), 0)
        - offset_vector,
        size=mp.Vector3(dpml + dbuffer, hSiN),
    )
)

# BOX
geometry.append(
    mp.Block(
        material=SiO2,
        center=mp.Vector3(0, -0.5 * (hSiN + hSiO2)) - offset_vector,
        size=mp.Vector3(mp.inf, hSiO2),
    )
)

# Substrate
geometry.append(
    mp.Block(
        material=Si,
        center=mp.Vector3(0, -0.5 * (hSiN + hSi + dpml + dbuffer) - hSiO2)
        - offset_vector,
        size=mp.Vector3(mp.inf, hSi + dpml + dbuffer),
    )
)

# PMLs
boundary_layers = [mp.PML(dpml)]

# Source

# mode frequency
fcen = 1 / 0.635

waveguide_port_center = mp.Vector3(-1 * dtaper, 0) - offset_vector
waveguide_port_size = mp.Vector3(0, 2 * haircore - 0.1)
fiber_port_center = (
    mp.Vector3(
        (0.5 * sz - dpml + y_offset - 1) * np.sin(fiber_angle) + fiber_xposition,
        0.5 * sz - dpml + y_offset - 1,
    )
    - offset_vector
)
fiber_port_size = mp.Vector3(sxy * 3 / 5 - 2 * dpml - 2, 0)

# Waveguide source
sources = [
    mp.EigenModeSource(
        src=mp.GaussianSource(fcen, fwidth=0.1 * fcen),
        size=waveguide_port_size,
        center=waveguide_port_center,
        eig_band=1,
        direction=mp.X,
        eig_match_freq=True,
        eig_parity=mp.ODD_Z,
    )
]

# Fiber source
# sources = [mp.EigenModeSource(src=mp.GaussianSource(fcen, fwidth=0.1*fcen),
#                               size=fiber_port_size,
#                               center=fiber_port_center,
#                               eig_band=1,
#                               direction=mp.NO_DIRECTION,
#                               eig_kpoint=mp.Vector3(y=-1).rotate(mp.Vector3(z=1), -1*fiber_angle),
#                               eig_match_freq=True,
#                               eig_parity=mp.ODD_Z)]

# symmetries = [mp.Mirror(mp.Y,-1)]
symmetries = []

sim = mp.Simulation(
    resolution=resolution,
    cell_size=cell_size,
    boundary_layers=boundary_layers,
    geometry=geometry,
    # geometry_center=mp.Vector3(x_offset, y_offset),
    sources=sources,
    dimensions=2,
    symmetries=symmetries,
    eps_averaging=False,
)

# Ports
waveguide_monitor_port = mp.ModeRegion(
    center=waveguide_port_center + mp.Vector3(x=0.2), size=waveguide_port_size
)
waveguide_monitor = sim.add_mode_monitor(
    fcen, 0, 1, waveguide_monitor_port, yee_grid=True
)
fiber_monitor_port = mp.ModeRegion(
    center=fiber_port_center - mp.Vector3(y=0.2),
    size=fiber_port_size,
    direction=mp.NO_DIRECTION,
)
fiber_monitor = sim.add_mode_monitor(fcen, 0, 1, fiber_monitor_port)

# nearfield = sim.add_near2far(fcen, 0, 1, mp.Near2FarRegion(mp.Vector3(x_offset,0.5*sz-dpml+y_offset)-offset_vector, size=mp.Vector3(sxy-2*dpml,0)))

In [ ]:
%%capture
sim.init_sim()

In [ ]:
f = plt.figure(dpi=100)
sim.plot2D(ax=f.gca())
plt.show()

In [ ]:
f = plt.figure(dpi=100)
animate = mp.Animate2D(sim, mp.Ez, f=f, normalize=True)
sim.run(mp.at_every(1, animate), until=100)
plt.close()

In [ ]:
animate.to_jshtml(10)

In [ ]:
import meep as mp
import math

resolution = 64  # pixels/unit length (1 um)

hSiN = 0.44
hSiO2 = 3.2
hSi = 1
hair = 4

dgrat = 0.6 * 0.310526
dgap = 0.6 * (1 - 0.310526)
a = dgrat + dgap
N = 20
N = N + 1
dtaper = 12

dbuffer = 0.5
dpml = 1

# Fiber parameters, from SMF-633-4/125-1-L or PMF-633-4/125-0.25-L
fiber_core = 4
fiber_clad = 120
fiber_angle = 8
fiber_angle = np.radians(fiber_angle)
fiber_xposition = 1
fiber_air_gap = 1
hfiber = 3
haircore = 2
hfiber_geom = 100  # Some large number to make fiber extend into PML
# Index from 0.10 @ 633 nm numerical aperture
# cladding 635 nm real index, From NA, n=1.4535642400664652
nClad = 1.4535642400664652
Clad = mp.Medium(index=nClad)
# Pure fused silica core core 635 nm real index (will be SiO2 below)

# MEEP's computational cell is always centered at (0,0), but code has beginning of grating at (0,0)
sxy = 2 * dpml + dtaper + a * N + 2 * dbuffer  # sx here
sz = 2 * dbuffer + hSiO2 + hSiN + hair + hSi + 2 * dpml  # sy here
# comp_origin_x = dpml + dbuffer + dtaper
comp_origin_x = 0
# meep_origin_x = sxy/2
# x_offset = meep_origin_x - comp_origin_x
x_offset = 0
# comp_origin_y = dpml + hSi + hSiO2 + hSiN/2
comp_origin_y = 0
# meep_origin_y = sz/2
# y_offset = meep_origin_y - comp_origin_y
y_offset = 0

# x_offset_vector = mp.Vector3(x_offset,0)
# offset_vector = mp.Vector3(x_offset, y_offset)
offset_vector = mp.Vector3(0, 0, 0)

# Si3N4 635 nm real index
nSiN = 2.0102
SiN = mp.Medium(index=nSiN)
# SiO2 635 nm real index
nSiO2 = 1.4569
SiO2 = mp.Medium(index=nSiO2)
# Si substrate 635 nm complex index, following https://meep.readthedocs.io/en/latest/Materials/#conductivity-and-complex
# eps = 15.044 + i*0.14910
Si = mp.Medium(epsilon=15.044, D_conductivity=2 * math.pi * 0.635 * 0.14910 / 15.044)

# We will do x-z plane simulation
cell_size = mp.Vector3(sxy, sz)

geometry = []

# Fiber (defined first to be overridden)

# Core
# fiber_offset = mp.Vector3(fiber_xposition + extrax, hSiN/2 + hair + haircore + extray) - offset_vector
geometry.append(
    mp.Block(
        material=Clad,
        center=mp.Vector3(x=fiber_xposition) - offset_vector,
        size=mp.Vector3(fiber_clad, hfiber_geom),
        e1=mp.Vector3(x=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
        e2=mp.Vector3(y=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
    )
)
geometry.append(
    mp.Block(
        material=SiO2,
        center=mp.Vector3(x=fiber_xposition) - offset_vector,
        size=mp.Vector3(fiber_core, hfiber_geom),
        e1=mp.Vector3(x=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
        e2=mp.Vector3(y=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
    )
)

# air
geometry.append(
    mp.Block(
        material=mp.air,
        center=mp.Vector3(0, haircore / 2) - offset_vector,
        size=mp.Vector3(mp.inf, haircore),
    )
)

# waveguide
geometry.append(
    mp.Block(
        material=SiN,
        center=mp.Vector3(0, 0) - offset_vector,
        size=mp.Vector3(mp.inf, hSiN),
    )
)

# grating etch
for n in range(0, N):
    geometry.append(
        mp.Block(
            material=mp.air,
            center=mp.Vector3(n * a + dgap / 2, 0) - offset_vector,
            size=mp.Vector3(dgap, hSiN),
        )
    )

geometry.append(
    mp.Block(
        material=mp.air,
        center=mp.Vector3(sxy - comp_origin_x - 0.5 * (dpml + dbuffer), 0)
        - offset_vector,
        size=mp.Vector3(dpml + dbuffer, hSiN),
    )
)

# BOX
geometry.append(
    mp.Block(
        material=SiO2,
        center=mp.Vector3(0, -0.5 * (hSiN + hSiO2)) - offset_vector,
        size=mp.Vector3(mp.inf, hSiO2),
    )
)

# Substrate
geometry.append(
    mp.Block(
        material=Si,
        center=mp.Vector3(0, -0.5 * (hSiN + hSi + dpml + dbuffer) - hSiO2)
        - offset_vector,
        size=mp.Vector3(mp.inf, hSi + dpml + dbuffer),
    )
)

# PMLs
boundary_layers = [mp.PML(dpml)]

# Source

# mode frequency
fcen = 1 / 0.635

waveguide_port_center = mp.Vector3(-1 * dtaper, 0) - offset_vector
waveguide_port_size = mp.Vector3(0, 2 * haircore - 0.1)
fiber_port_center = (
    mp.Vector3(
        (0.5 * sz - dpml + y_offset - 1) * np.sin(fiber_angle) + fiber_xposition,
        0.5 * sz - dpml + y_offset - 1,
    )
    - offset_vector
)
fiber_port_size = mp.Vector3(sxy * 3 / 5 - 2 * dpml - 2, 0)

# Waveguide source
# sources = [mp.EigenModeSource(src=mp.GaussianSource(fcen, fwidth=0.1*fcen),
#                               size=waveguide_port_size,
#                               center=waveguide_port_center,
#                               eig_band=1,
#                               direction=mp.X,
#                               eig_match_freq=True,
#                               eig_parity=mp.ODD_Z)]

# Fiber source
sources = [
    mp.EigenModeSource(
        src=mp.GaussianSource(fcen, fwidth=0.1 * fcen),
        size=fiber_port_size,
        center=fiber_port_center,
        eig_band=1,
        direction=mp.NO_DIRECTION,
        eig_kpoint=mp.Vector3(y=-1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
        eig_match_freq=True,
        eig_parity=mp.ODD_Z,
    )
]

# symmetries = [mp.Mirror(mp.Y,-1)]
symmetries = []

sim = mp.Simulation(
    resolution=resolution,
    cell_size=cell_size,
    boundary_layers=boundary_layers,
    geometry=geometry,
    # geometry_center=mp.Vector3(x_offset, y_offset),
    sources=sources,
    dimensions=2,
    symmetries=symmetries,
    eps_averaging=False,
)

# Ports
waveguide_monitor_port = mp.ModeRegion(
    center=waveguide_port_center + mp.Vector3(x=0.2), size=waveguide_port_size
)
waveguide_monitor = sim.add_mode_monitor(
    fcen, 0, 1, waveguide_monitor_port, yee_grid=True
)
fiber_monitor_port = mp.ModeRegion(
    center=fiber_port_center - mp.Vector3(y=0.2),
    size=fiber_port_size,
    direction=mp.NO_DIRECTION,
)
fiber_monitor = sim.add_mode_monitor(fcen, 0, 1, fiber_monitor_port)

# nearfield = sim.add_near2far(fcen, 0, 1, mp.Near2FarRegion(mp.Vector3(x_offset,0.5*sz-dpml+y_offset)-offset_vector, size=mp.Vector3(sxy-2*dpml,0)))

In [ ]:
%%capture
sim.init_sim()

In [ ]:
f = plt.figure(dpi=100)
sim.plot2D(ax=f.gca())
plt.show()

In [ ]:
f = plt.figure(dpi=100)
animate = mp.Animate2D(sim, mp.Ez, f=f, normalize=True)
sim.run(mp.at_every(1, animate), until=100)
plt.close()

In [ ]:
animate.to_jshtml(10)

In [ ]:
import meep as mp
import math

resolution = 64  # pixels/unit length (1 um)

hSiN = 0.44
hSiO2 = 3.2
hSi = 1
hair = 4

dgrat = 0.6 * 0.310526
dgap = 0.6 * (1 - 0.310526)
a = dgrat + dgap
N = 20
N = N + 1
dtaper = 12

dbuffer = 0.5
dpml = 1

# Fiber parameters, from SMF-633-4/125-1-L or PMF-633-4/125-0.25-L
fiber_core = 4
fiber_clad = 120
fiber_angle = 8
fiber_angle = np.radians(fiber_angle)
fiber_xposition = 1
fiber_air_gap = 1
hfiber = 3
haircore = 2
hfiber_geom = 100  # Some large number to make fiber extend into PML
# Index from 0.10 @ 633 nm numerical aperture
# cladding 635 nm real index, From NA, n=1.4535642400664652
nClad = 1.4535642400664652
Clad = mp.Medium(index=nClad)
# Pure fused silica core core 635 nm real index (will be SiO2 below)

# MEEP's computational cell is always centered at (0,0), but code has beginning of grating at (0,0)
sxy = 2 * dpml + dtaper + a * N + 2 * dbuffer  # sx here
sz = 2 * dbuffer + hSiO2 + hSiN + hair + hSi + 2 * dpml  # sy here
# comp_origin_x = dpml + dbuffer + dtaper
comp_origin_x = 0
# meep_origin_x = sxy/2
# x_offset = meep_origin_x - comp_origin_x
x_offset = 0
# comp_origin_y = dpml + hSi + hSiO2 + hSiN/2
comp_origin_y = 0
# meep_origin_y = sz/2
# y_offset = meep_origin_y - comp_origin_y
y_offset = 0

# x_offset_vector = mp.Vector3(x_offset,0)
# offset_vector = mp.Vector3(x_offset, y_offset)
offset_vector = mp.Vector3(0, 0, 0)

# Si3N4 635 nm real index
nSiN = 2.0102
SiN = mp.Medium(index=nSiN)
# SiO2 635 nm real index
nSiO2 = 1.4569
SiO2 = mp.Medium(index=nSiO2)
# Si substrate 635 nm complex index, following https://meep.readthedocs.io/en/latest/Materials/#conductivity-and-complex
# eps = 15.044 + i*0.14910
Si = mp.Medium(epsilon=15.044, D_conductivity=2 * math.pi * 0.635 * 0.14910 / 15.044)

# We will do x-z plane simulation
cell_size = mp.Vector3(sxy, sz)

geometry = []

# Fiber (defined first to be overridden)

# Core
# fiber_offset = mp.Vector3(fiber_xposition + extrax, hSiN/2 + hair + haircore + extray) - offset_vector
geometry.append(
    mp.Block(
        material=Clad,
        center=mp.Vector3(x=fiber_xposition) - offset_vector,
        size=mp.Vector3(fiber_clad, hfiber_geom),
        e1=mp.Vector3(x=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
        e2=mp.Vector3(y=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
    )
)
geometry.append(
    mp.Block(
        material=SiO2,
        center=mp.Vector3(x=fiber_xposition) - offset_vector,
        size=mp.Vector3(fiber_core, hfiber_geom),
        e1=mp.Vector3(x=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
        e2=mp.Vector3(y=1).rotate(mp.Vector3(z=1), -1 * fiber_angle),
    )
)

# air
geometry.append(
    mp.Block(
        material=mp.air,
        center=mp.Vector3(0, haircore / 2) - offset_vector,
        size=mp.Vector3(mp.inf, haircore),
    )
)

# waveguide
geometry.append(
    mp.Block(
        material=SiN,
        center=mp.Vector3(0, 0) - offset_vector,
        size=mp.Vector3(mp.inf, hSiN),
    )
)

# grating etch
for n in range(0, N):
    geometry.append(
        mp.Block(
            material=mp.air,
            center=mp.Vector3(n * a + dgap / 2, 0) - offset_vector,
            size=mp.Vector3(dgap, hSiN),
        )
    )

geometry.append(
    mp.Block(
        material=mp.air,
        center=mp.Vector3(sxy - comp_origin_x - 0.5 * (dpml + dbuffer), 0)
        - offset_vector,
        size=mp.Vector3(dpml + dbuffer, hSiN),
    )
)

# BOX
geometry.append(
    mp.Block(
        material=SiO2,
        center=mp.Vector3(0, -0.5 * (hSiN + hSiO2)) - offset_vector,
        size=mp.Vector3(mp.inf, hSiO2),
    )
)

# Substrate
geometry.append(
    mp.Block(
        material=Si,
        center=mp.Vector3(0, -0.5 * (hSiN + hSi + dpml + dbuffer) - hSiO2)
        - offset_vector,
        size=mp.Vector3(mp.inf, hSi + dpml + dbuffer),
    )
)

# PMLs
boundary_layers = [mp.PML(dpml)]

# Source

# mode frequency
fcen = 1 / 0.635

waveguide_port_center = mp.Vector3(-1 * dtaper, 0) - offset_vector
waveguide_port_size = mp.Vector3(0, 2 * haircore - 0.1)
fiber_port_center = (
    mp.Vector3(
        (0.5 * sz - dpml + y_offset - 1) * np.sin(fiber_angle) + fiber_xposition,
        0.5 * sz - dpml + y_offset - 1,
    )
    - offset_vector
)
fiber_port_size = mp.Vector3(sxy * 3 / 5 - 2 * dpml - 2, 0)

# Waveguide source
sources = [
    mp.EigenModeSource(
        src=mp.GaussianSource(fcen, fwidth=0.1 * fcen),
        size=waveguide_port_size,
        center=waveguide_port_center,
        eig_band=1,
        direction=mp.X,
        eig_match_freq=True,
        eig_parity=mp.ODD_Z,
    )
]

# # Fiber source
# sources = [mp.EigenModeSource(src=mp.GaussianSource(fcen, fwidth=0.1*fcen),
#                               size=fiber_port_size,
#                               center=fiber_port_center,
#                               eig_band=1,
#                               direction=mp.NO_DIRECTION,
#                               eig_kpoint=mp.Vector3(y=-1).rotate(mp.Vector3(z=1), -1*fiber_angle),
#                               eig_match_freq=True,
#                               eig_parity=mp.ODD_Z)]

# symmetries = [mp.Mirror(mp.Y,-1)]
symmetries = []

sim = mp.Simulation(
    resolution=resolution,
    cell_size=cell_size,
    boundary_layers=boundary_layers,
    geometry=geometry,
    # geometry_center=mp.Vector3(x_offset, y_offset),
    sources=sources,
    dimensions=2,
    symmetries=symmetries,
    eps_averaging=False,
)

# Ports
waveguide_monitor_port = mp.ModeRegion(
    center=waveguide_port_center + mp.Vector3(x=0.2), size=waveguide_port_size
)
waveguide_monitor = sim.add_mode_monitor(
    fcen, 0, 1, waveguide_monitor_port, yee_grid=True
)
fiber_monitor_port = mp.ModeRegion(
    center=fiber_port_center - mp.Vector3(y=0.2),
    size=fiber_port_size,
    direction=mp.NO_DIRECTION,
)
fiber_monitor = sim.add_mode_monitor(fcen, 0, 1, fiber_monitor_port)

# nearfield = sim.add_near2far(fcen, 0, 1, mp.Near2FarRegion(mp.Vector3(x_offset,0.5*sz-dpml+y_offset)-offset_vector, size=mp.Vector3(sxy-2*dpml,0)))

In [ ]:
%%capture
sim.init_sim()

In [ ]:
f = plt.figure(dpi=100)
sim.plot2D(ax=f.gca())
plt.show()

In [ ]:
f = plt.figure(dpi=100)
animate = mp.Animate2D(sim, mp.Ez, f=f, normalize=True)
sim.run(mp.at_every(1, animate), until=100)
plt.close()

In [ ]:
animate.to_jshtml(10)

In [ ]:
neff = 1.93 * 0.310526 + (1 - 0.310526)
neff

In [ ]:
635 / (neff - np.sin(8 * np.pi / 180))

What we're seeing is probably not good directionality.

In [ ]:
np.where((np.abs(df["Prs"].to_numpy() - 0.18057623742207388) < 0.01))

In [ ]:
df.iloc[[26778, 26779, 75359, 75360, 75361]]

In [ ]:
df.loc[df["Prs"] == np.max(df["Prs"].to_numpy())]